In [3]:
def circular_conv(image, kernel, linear=False):
    # Get the dimensions of the input image and kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape
    
    # Flip the kernel
    kernel = np.flipud(np.fliplr(kernel))
    
    # Create an empty array to store the convolution result
    result = np.zeros_like(image)
    
    # Calculate the amount of padding required based on the kernel size
    padding_height = kernel_height // 2
    padding_width = kernel_width // 2
    
    # Check if circular or linear convolution is requested
    if not linear:
        # Circular convolution
        # Pad the image using the 'wrap' mode to handle circular convolution
        padded_image = np.pad(image, ((padding_height, padding_height), (padding_width, padding_width)), mode='wrap')
        
        # Iterate over each pixel in the input image
        for i in range(image_height):
            for j in range(image_width):
                # Extract the region of interest from the padded image
                roi = padded_image[i:i+kernel_height, j:j+kernel_width]
                
                # Perform element-wise multiplication between the kernel and the ROI
                conv_result = np.sum(roi * kernel)
                
                # Store the result in the output array
                result[i, j] = conv_result
                
    else:
        # Linear convolution (with zero padding)
        # Pad the image using the 'constant' mode to handle linear convolution
        padded_image = np.pad(image, ((padding_height, padding_height), (padding_width, padding_width)), mode='constant')
        
        # Iterate over each pixel in the input image
        for i in range(image_height):
            for j in range(image_width):
                # Extract the region of interest from the padded image
                roi = padded_image[i:i+kernel_height, j:j+kernel_width]
                
                # Perform element-wise multiplication between the kernel and the ROI
                conv_result = np.sum(roi * kernel)
                
                # Store the result in the output array
                result[i, j] = conv_result
    
    return result

# Create sample image and kernel
image = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])
kernel = np.array([[1, 5],
                   [3, 4]])

# Perform circular convolution
circular_result = circular_conv(image, kernel)
linear_result = circular_conv(image, kernel, linear=True)

print("Circularly Convolved Result:")
print(circular_result)
print("\nLinearly Convolved Result:")
print(linear_result)


Circularly Convolved Result:
[[73 59 72]
 [49 35 48]
 [88 74 87]]

Linearly Convolved Result:
[[ 1  7 13]
 [ 7 35 48]
 [19 74 87]]


In [2]:
import numpy as np

def DFT(I):
    N, M = I.shape[0], I.shape[1]
    
    row_ind = np.arange(N).reshape(N, 1)
    col_ind = np.arange(M).reshape(1, M)
                
    c1 = np.exp(-2 * np.pi * 1j / N)
    dft_matrix_row = c1 ** (row_ind * col_ind)
            
    c2 = np.exp(-2 * np.pi * 1j / M)
    dft_matrix_col = c2 ** (row_ind * col_ind)
              
    return np.dot(np.dot(dft_matrix_row, I), dft_matrix_col)

def IDFT(I_hat):
    N, M = I_hat.shape[0], I_hat.shape[1]
    
    row_ind = np.arange(N).reshape(N, 1)
    col_ind = np.arange(M).reshape(1, M)
                
    c1 = np.exp(2 * np.pi * 1j / N)
    idft_matrix_row = c1 ** (row_ind * col_ind)
            
    c2 = np.exp(2 * np.pi * 1j / M)
    idft_matrix_col = c2 ** (row_ind * col_ind)
            
    I = np.dot(np.dot(idft_matrix_row, I_hat), idft_matrix_col)
    I = I / (N * M)
    
    return I.astype(np.int64)

def circular_conv_via_DFT(image, kernel):
    # Get sizes
    N, M = image.shape[0], image.shape[1]
    n, m = kernel.shape[0], kernel.shape[1]
    
    # Calculate padding required for circular convolution
    pad_width = ((0, N - n), (0, M - m))

    # Pad the kernel to match the size of the image
    padded_kernel = np.pad(kernel, pad_width, mode='wrap')
    print(padded_kernel)
    
    # Calculate DFT of image and padded kernel
    image_dft = DFT(image)
    kernel_dft = DFT(padded_kernel)
    
    # Element-wise multiplication
    product_dft = image_dft * kernel_dft
    
    # Calculate IDFT of the product
    result_conv = IDFT(product_dft)
    
    return result_conv

# Example usage
image = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])
kernel = np.array([[1, 5],
                   [3, 4]])

result = circular_conv_via_DFT(image, kernel)
print("Result of circular convolution via DFT:")
print(result)


[[1 5 1]
 [3 4 3]
 [1 5 1]]
Result of circular convolution via DFT:
[[138 120 129]
 [120 102 110]
 [129 111 120]]


C:\Users\user\AppData\Local\Temp\ipykernel_4584\1232414322.py:32: ComplexWarning: Casting complex values to real discards the imaginary part
  return I.astype(np.int64)


In [4]:
from scipy.fft import fft2, ifft2

def circular_convolution(image, kernel):
    # Apply FFT to image and kernel
    fft_image = fft2(image)
    fft_kernel = fft2(kernel, s=image.shape)

    # Perform element-wise multiplication in frequency domain
    fft_convolved = fft_image * fft_kernel

    # Compute inverse FFT to get circular convolution
    convolved_image = np.real(ifft2(fft_convolved))

    return convolved_image